In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
archivo="/content/drive/MyDrive/used_cars.xlsx"
df=pd.read_excel(archivo)

df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,51000,E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,10300
1,Hyundai,Palisade SEL,2021,34742,Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,38005
2,Lexus,RX 350 RX 350,2022,22372,Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,54598
3,INFINITI,Q50 Hybrid Sport,2015,88900,Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,15500
4,Audi,Q3 45 S line Premium Plus,2021,9835,Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,34999


In [ ]:

# 1. Limpieza y filtrado de datos
print("Filas originales:", len(df))

# Eliminar filas con valores críticos faltantes
df = df.dropna(subset=['price', 'brand', 'model_year', 'milage', 'engine'])

# Eliminar outliers en precio (quitamos el 5% superior e inferior)
q_low = df['price'].quantile(0.05)
q_high = df['price'].quantile(0.95)
df = df[(df['price'] >= q_low) & (df['price'] <= q_high)]

# Eliminar outliers en millaje
q_low_milage = df['milage'].quantile(0.05)
q_high_milage = df['milage'].quantile(0.95)
df = df[(df['milage'] >= q_low_milage) & (df['milage'] <= q_high_milage)]

print("Filas después de limpieza:", len(df))

# 2. Feature engineering
# Extraer horsepower
def extract_hp(engine_str):
    if isinstance(engine_str, str):
        match = re.search(r'(\d+\.\d+|\d+)HP', engine_str)
        if match:
            return float(match.group(1))
    return np.nan

df['horsepower'] = df['engine'].apply(extract_hp)
df = df.dropna(subset=['horsepower'])

# Extraer desplazamiento del motor
def extract_displacement(engine_str):
    if isinstance(engine_str, str):
        match = re.search(r'(\d+\.\d+|\d+)L', engine_str)
        if match:
            return float(match.group(1))
    return np.nan

df['displacement'] = df['engine'].apply(extract_displacement)
df = df.dropna(subset=['displacement'])

# Crear características adicionales (logra mejor relación con el precio a predecir)
df['vehicle_age'] = 2023 - df['model_year']  # Ajustar año actual si es necesario
df['milage_per_year'] = df['milage'] / (df['vehicle_age'] + 1)  # +1 para evitar división por cero
df['power_to_weight_ratio'] = df['horsepower'] / df['displacement']

# Agrupar marcas poco frecuentes
brand_counts = df['brand'].value_counts()
infrequent_brands = brand_counts[brand_counts < 10].index
df['brand_grouped'] = df['brand'].apply(lambda x: 'Other' if x in infrequent_brands else x)

# 3. Selección de características
features = [
    'vehicle_age',
    'milage',
    'horsepower',
    'displacement',
    'milage_per_year',
    'power_to_weight_ratio',
    'brand_grouped',
    'fuel_type'
]

X = df[features]
y = df['price']

# Codificar variables categóricas
X = pd.get_dummies(X, columns=['brand_grouped', 'fuel_type'], drop_first=True)

# 4. División de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. Entrenamiento del modelo con ajuste de hiperparámetros
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf_model = RandomForestRegressor(random_state=42, n_jobs=-1)
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

# Mejor modelo
best_model = grid_search.best_estimator_

# 6. Evaluación
y_pred = best_model.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Mejores parámetros: {grid_search.best_params_}")
print(f"R²: {r2:.4f}")
print(f"RMSE: {rmse:.2f}")



Filas originales: 4009
Filas después de limpieza: 3247
Mejores parámetros: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
R²: 0.8184
RMSE: 8631.36
